In [72]:
import duckdb

In [73]:
# creates the database and loads the spatial extension
con = duckdb.connect(database='../database.duckdb')
con.sql('INSTALL spatial; LOAD spatial')

In [74]:
# creates the courts view, taking advantage of the spatial extension and the EXCLUDE syntax sugar
con.sql('''
CREATE OR REPLACE VIEW courts AS
SELECT
    * EXCLUDE (geometry),
    ST_GeomFromWKB(geometry) AS GEOMETRY
FROM
    '../Courts.parquet';
SELECT * FROM courts LIMIT 5;
''')

┌──────────┬──────────────────────┬─────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ OBJECTID │       GLOBALID       │    ASSET_ID     │ … │ CREATED_USER │ UPDATED_USER │       GEOMETRY       │
│  int64   │       varchar        │     varchar     │   │   varchar    │   varchar    │       geometry       │
├──────────┼──────────────────────┼─────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│      125 │ {EF8386EA-B072-4EF…  │ MC-D54-COURT-02 │ … │ NULL         │ parksparks   │ MULTIPOLYGON (((13…  │
│      124 │ {70B2E5F8-7C72-4A9…  │ MC-D54-COURT-01 │ … │ NULL         │ parksparks   │ MULTIPOLYGON (((13…  │
│      545 │ {9A9E1645-D545-4E6…  │ MC-D47-COURT-01 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
│      546 │ {F5DF1C67-11B3-4BD…  │ MC-D47-COURT-04 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
│      547 │ {5A6DC082-71DE-46A…  │ MC-D47-COURT-05 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
├

In [75]:
# creates the parks view, taking advantage of the spatial extension and the EXCLUDE syntax sugar

con.sql('''
CREATE OR REPLACE VIEW parks AS
SELECT
    * EXCLUDE (geometry),
    ST_GeomFromWKB(geometry) AS GEOMETRY
FROM
    '../ParkUnits.parquet';
SELECT * FROM parks LIMIT 5;
''')

┌──────────┬──────────────────────┬─────────┬──────────┬───┬──────────────────────┬───────────┬──────────────────────┐
│ OBJECTID │      PARK_NAME       │  OWNER  │  STATUS  │ … │      GLOBALID_1      │ PARK_CODE │       GEOMETRY       │
│  int64   │       varchar        │ varchar │ varchar  │   │       varchar        │  varchar  │       geometry       │
├──────────┼──────────────────────┼─────────┼──────────┼───┼──────────────────────┼───────────┼──────────────────────┤
│      649 │ Watts Branch SVU 4   │ M-NCPPC │ Existing │ … │ {C109BC29-B711-47B…  │ P67       │ MULTIPOLYGON (((12…  │
│      650 │ Muddy Branch SVU 2   │ M-NCPPC │ Existing │ … │ {668B3CF8-116F-41A…  │ P28       │ MULTIPOLYGON (((12…  │
│      737 │ Quince Orchard Val…  │ M-NCPPC │ Existing │ … │ {CE19FC8B-A5D1-45D…  │ B46       │ MULTIPOLYGON (((12…  │
│      738 │ Fleming Local Park   │ M-NCPPC │ Existing │ … │ {A11112E7-2EF4-406…  │ D41       │ MULTIPOLYGON (((12…  │
│      816 │ Veirs Mill Local P…  │ M-NCPPC │ Ex

In [76]:
# returns the columns of the courts view that are of type INT, which most happen to be one hot encoded sports to be used as a reference for the parks_enriched table
con.sql('''
SELECT
    *
FROM
    (DESCRIBE courts)
WHERE
    column_type LIKE 'INT%'
''')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ BASKETBALL   │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ BOCCI        │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ FUTSAL       │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ HANDBALL     │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ HOCKEY       │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ PADDLEBALL   │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ PICKLEBALL   │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ SHUFFLEBOARD │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ RACQUETBALL  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ TAICHI       │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ TENN

In [78]:
# creates the parks_enriched table, taking advantage of the spatial extension and the EXCLUDE syntax sugar
con.sql('''
CREATE OR REPLACE TABLE parks_enriched AS
SELECT
    parks.*,
    COUNT(courts.*) AS courts_count,
    -- counts of sport courts available at the parks
    CAST(COALESCE(BASKETBALL.SUM(), 0) AS INT) AS basketball_count, -- .SUM() take advantage
    CAST(COALESCE(BOCCI.SUM(), 0) AS INT) AS bocci_count,
    CAST(COALESCE(FUTSAL.SUM(), 0) AS INT) AS futsal_count,
    CAST(COALESCE(HANDBALL.SUM(), 0) AS INT) AS handball_count,
    CAST(COALESCE(HOCKEY.SUM(), 0) AS INT) AS hockey_count,
    CAST(COALESCE(PADDLEBALL.SUM(), 0) AS INT) AS paddleball_count,
    CAST(COALESCE(PICKLEBALL.SUM(), 0) AS INT) AS pickleball_count,
    CAST(COALESCE(SHUFFLEBOARD.SUM(), 0) AS INT) AS shuffleboard_count,
    CAST(COALESCE(RACQUETBALL.SUM(), 0) AS INT) AS racquetball_count,
    CAST(COALESCE(TAICHI.SUM(), 0) AS INT) AS taichi_count,
    CAST(COALESCE(TENNIS.SUM(), 0) AS INT) AS tennis_count,
    CAST(COALESCE(VOLLEYBALL.SUM(), 0) AS INT) AS volleyball_count
FROM
    parks
LEFT JOIN courts ON
    ST_Contains(parks.geometry, courts.geometry) -- spatial join
WHERE
    parks.owner = 'M-NCPPC'
    AND parks.status = 'Existing'
GROUP BY parks.* -- group by all parks fields with wildcard
ORDER BY courts_count DESC;
SELECT * FROM parks_enriched LIMIT 5;
''')

┌──────────┬──────────────────────┬─────────┬───┬───────────────────┬──────────────┬──────────────┬──────────────────┐
│ OBJECTID │      PARK_NAME       │  OWNER  │ … │ racquetball_count │ taichi_count │ tennis_count │ volleyball_count │
│  int64   │       varchar        │ varchar │   │       int32       │    int32     │    int32     │      int32       │
├──────────┼──────────────────────┼─────────┼───┼───────────────────┼──────────────┼──────────────┼──────────────────┤
│      218 │ Olney Manor Recrea…  │ M-NCPPC │ … │                 1 │            0 │           18 │                2 │
│      731 │ Cabin John Regiona…  │ M-NCPPC │ … │                 0 │            1 │           15 │                2 │
│      853 │ Wheaton Regional P…  │ M-NCPPC │ … │                 0 │            0 │           12 │                0 │
│       27 │ Fairland Recreatio…  │ M-NCPPC │ … │                 0 │            0 │            6 │                0 │
│      408 │ Norwood Local Park   │ M-NCPPC │ … 

In [79]:
con.sql('''
COPY (SELECT * FROM parks_enriched) TO '../parks_enriched.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');
''')

In [80]:
con.close()